In [ ]:
import numpy as np
import qasm_reader
import os
import matplotlib.pyplot as plt

SAVE_PATH = "path/to/plots" # Replace this with your path

# Download the QASMBench repository from https://github.com/pnnl/QASMBench

BENCH_PATH = "path/to/qasmbench_benchmarks" # Replace this with the QASMBench path

FILES = {}

for f in os.listdir(BENCH_PATH):
    if f.endswith(".qasm"):
        name = os.path.splitext(f)[0]  # filename without extension
        FILES[name] = os.path.join(BENCH_PATH, f)

def qasm_file_to_interactions(file_path):
    qasm_file = qasm_reader.read_QASM_file(file_path)[4:]

    interactions = []

    for line in qasm_file:
        out = qasm_reader.read_QASM_line(line)
        
        if out:
            gate, targs = out
            # if len(targs) > 2: 
            #     raise ValueError("Too many targets for a 2-qubit gate")
            if len(targs) == 2:
                interactions.append((min(targs[0], targs[1]), max(targs[0], targs[1])))
    
    return interactions

def interactions_to_memaccess(interactions):
    num_qubits = max([max(interaction) for interaction in interactions]) + 1
    
    full_mat = np.zeros((len(interactions), num_qubits))

    row_idx = 0
    for interaction in interactions:    
        # see if we can schedule this interaction without overlap
        if full_mat[row_idx, interaction[0]] == 0 and full_mat[row_idx, interaction[1]] == 0:
            full_mat[row_idx, interaction[0]] = 1
            full_mat[row_idx, interaction[1]] = 1
        else:
            # we need to add a new row
            row_idx += 1
            full_mat[row_idx, interaction[0]] = 1
            full_mat[row_idx, interaction[1]] = 1
            

    full_mat = full_mat[:row_idx+1, :]
    return full_mat

def plot_memory_access(schedule_mat, filename):
    # Transpose so that the x-axis corresponds to cycles and y-axis to qubits.
    image = schedule_mat.T

    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Set origin to 'upper' so that qubit 0 is at the top.
    im = ax.imshow(image, cmap='hot', interpolation='nearest', aspect='auto', origin='upper')
    ax.set_xlabel("Cycle")
    ax.set_ylabel("Qubit")
    ax.set_title("Randomized Memory Access " + filename)

    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_PATH, f"{filename}.png"), dpi=600)
    plt.show()

In [ ]:
for filename in FILES:
    FILE_TO_PROCESS = FILES[filename]
    interactions = qasm_file_to_interactions(FILE_TO_PROCESS)
    schedule_mat = interactions_to_memaccess(interactions)

    # plot_memory_access_with_initial_window(schedule_mat, dx=5, dy=16)
    plot_memory_access(schedule_mat, filename=filename)